In [9]:
!pip install ctranslate2
!pip install py3nvml

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [10]:
from transformers import PyTorchBenchmark, PyTorchBenchmarkArguments

args = PyTorchBenchmarkArguments(models=["google/flan-t5-large"], batch_sizes=[8], sequence_lengths=[8, 32, 128, 512])
benchmark = PyTorchBenchmark(args)

/home/meeami/anaconda3/envs/text-generation-inference/lib/python3.9/site-packages/transformers/benchmark/benchmark_args_utils.py:136: FutureWarning: The class <class 'transformers.benchmark.benchmark_args.PyTorchBenchmarkArguments'> is deprecated. Hugging Face Benchmarking utils are deprecated in general and it is advised to use external Benchmarking libraries  to benchmark Transformer models.
  warnings.warn(
/home/meeami/anaconda3/envs/text-generation-inference/lib/python3.9/site-packages/transformers/benchmark/benchmark_utils.py:615: FutureWarning: The class <class 'transformers.benchmark.benchmark.PyTorchBenchmark'> is deprecated. Hugging Face Benchmarking utils are deprecated in general and it is advised to use external Benchmarking libraries  to benchmark Transformer models.
  warnings.warn(


In [11]:
results = benchmark.run()
print(results)

1 / 1

====================       INFERENCE - SPEED - RESULT       ====================
--------------------------------------------------------------------------------
          Model Name             Batch Size     Seq Length     Time in s   
--------------------------------------------------------------------------------
     google/flan-t5-large            8               8             0.033     
     google/flan-t5-large            8               32            0.078     
     google/flan-t5-large            8              128            0.252     
     google/flan-t5-large            8              512            1.188     
--------------------------------------------------------------------------------

====================      INFERENCE - MEMORY - RESULT       ====================
--------------------------------------------------------------------------------
          Model Name             Batch Size     Seq Length    Memory in MB 
------------------------------------------

# FlanT5-Large (original)

In [12]:
import time
import numpy as np

from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-large")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-large").cpu()

input_text = "translate English to German: How old are you?"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids

outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0]))


<pad> Wie alte sind Sie?</s>


In [13]:
import tqdm
seq_len = [8, 32, 128, 512]
for i in range(4):
    start = time.time()
    for j in tqdm.tqdm(range(100)):
        input_text = "translate English to German: " + "house "* seq_len[i]
        input_ids = tokenizer(input_text, return_tensors="pt").input_ids
        outputs = model.generate(input_ids)    
        results = tokenizer.decode(outputs[0]) 
    end = time.time()
    print("seq_len: ", seq_len[i], "time: ", (end - start)/100)


    

100%|██████████| 100/100 [01:35<00:00,  1.05it/s]


seq_len:  8 time:  0.9559398126602173


100%|██████████| 100/100 [02:41<00:00,  1.61s/it]


seq_len:  32 time:  1.6130975890159607


100%|██████████| 100/100 [02:54<00:00,  1.74s/it]


seq_len:  128 time:  1.743881630897522


100%|██████████| 100/100 [04:30<00:00,  2.71s/it]

seq_len:  512 time:  2.7074382948875426


# FLanT5-large ctranslate2(CPU)

In [14]:
!ct2-transformers-converter --model google/flan-t5-large --output_dir google/flan-t5-large-ct2

Traceback (most recent call last):
  File "/home/meeami/anaconda3/envs/text-generation-inference/bin/ct2-transformers-converter", line 8, in <module>
    sys.exit(main())
  File "/home/meeami/anaconda3/envs/text-generation-inference/lib/python3.9/site-packages/ctranslate2/converters/transformers.py", line 1595, in main
    converter.convert_from_args(args)
  File "/home/meeami/anaconda3/envs/text-generation-inference/lib/python3.9/site-packages/ctranslate2/converters/converter.py", line 50, in convert_from_args
    return self.convert(
  File "/home/meeami/anaconda3/envs/text-generation-inference/lib/python3.9/site-packages/ctranslate2/converters/converter.py", line 84, in convert
    raise RuntimeError(
RuntimeError: output directory google/flan-t5-large-ct2 already exists, use --force to override


In [2]:
import ctranslate2
import transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-large") 
translator = ctranslate2.Translator("google/flan-t5-large-ct2",device="cpu")
#print(translator.intra_threads,translator.inter_threads )


input_text = "translate English to German: The house is wonderful."
input_tokens = tokenizer.convert_ids_to_tokens(tokenizer.encode(input_text))

results = translator.translate_batch([input_tokens])

output_tokens = results[0].hypotheses[0]
output_text = tokenizer.decode(tokenizer.convert_tokens_to_ids(output_tokens))

print(output_text)



/home/meeami/anaconda3/envs/text-generation-inference/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-07-07 02:31:47.581320: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Das Haus ist wunderbar.


In [3]:
import tqdm
import time
seq_len = [8, 32, 128, 512]
for i in range(4):
    start = time.time()
    for j in tqdm.tqdm(range(100)):
        input_text =input_text = "translate English to German: " + "house "* seq_len[i]
        input_tokens = tokenizer.convert_ids_to_tokens(tokenizer.encode(input_text))
        results = translator.translate_batch([input_tokens])
        output_tokens = results[0].hypotheses[0]
        output_text = tokenizer.decode(tokenizer.convert_tokens_to_ids(output_tokens))
    end = time.time()
    print("seq_len: ", seq_len[i], "time: ", (end - start)/100)


    

100%|██████████| 100/100 [01:03<00:00,  1.58it/s]


seq_len:  8 time:  0.6320943832397461


100%|██████████| 100/100 [02:46<00:00,  1.67s/it]


seq_len:  32 time:  1.6678615713119507


  7%|▋         | 7/100 [01:54<25:31, 16.47s/it]

# FLanT5-large ctranslate2(GPU)

In [1]:
import ctranslate2
import transformers
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-large") 
translator = ctranslate2.Translator("google/flan-t5-large-ct2",device="cuda")


input_text = "translate English to German: The house is wonderful."
input_tokens = tokenizer.convert_ids_to_tokens(tokenizer.encode(input_text))

results = translator.translate_batch([input_tokens])

output_tokens = results[0].hypotheses[0]
output_text = tokenizer.decode(tokenizer.convert_tokens_to_ids(output_tokens))

print(output_text)


/home/meeami/anaconda3/envs/text-generation-inference/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-07-07 02:37:58.338289: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Das Haus ist wunderbar.


In [2]:
import tqdm
import time
seq_len = [8, 32, 128, 512]
for i in range(4):
    start = time.time()
    for j in tqdm.tqdm(range(100)):
        input_text =input_text = "translate English to German: " + "house "* seq_len[i]
        input_tokens = tokenizer.convert_ids_to_tokens(tokenizer.encode(input_text))
        results = translator.translate_batch([input_tokens])
        output_tokens = results[0].hypotheses[0]
        output_text = tokenizer.decode(tokenizer.convert_tokens_to_ids(output_tokens))
    end = time.time()
    print("seq_len: ", seq_len[i], "time: ", (end - start)/100)


100%|██████████| 100/100 [00:06<00:00, 14.68it/s]


seq_len:  8 time:  0.06823009014129638


100%|██████████| 100/100 [00:21<00:00,  4.71it/s]


seq_len:  32 time:  0.21241705179214476


100%|██████████| 100/100 [03:31<00:00,  2.12s/it]


seq_len:  128 time:  2.1185588908195494


100%|██████████| 100/100 [03:46<00:00,  2.26s/it]

seq_len:  512 time:  2.2636704897880553
